In [3]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [4]:
CUDA=torch.cuda.is_available()
device = torch.device("cuda" if CUDA else  "cpu")

In [5]:
device

device(type='cpu')

In [6]:
lines_filepath=os.path.join("cornell movie-dialogs corpus","movie_lines.txt")
conv_filepath = os.path.join("cornell movie-dialogs corpus","movie_conversations.txt")

In [7]:
lines_filepath

'cornell movie-dialogs corpus\\movie_lines.txt'

In [8]:
conv_filepath

'cornell movie-dialogs corpus\\movie_conversations.txt'

In [9]:
with open(lines_filepath,'r') as file:
    lines = file.readlines()

In [15]:
for line in lines[:8]:
    print(line.strip())

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No


In [30]:
with open(conv_filepath,'r') as file:
    lines = file.readlines()
for line in lines[:8]:
    print(line.strip())

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']


In [42]:
l = "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']"
eval(l.split(" +++$+++ ")[-1])

['L194', 'L195', 'L196', 'L197']

In [48]:
line_fields = ['lineID','characterID','movieID','character','text']
lines={}
with open(lines_filepath,'r') as file:
    for line in file:
        values = line.split(" +++$+++ ")
        lineObj = {}
        for i,field in enumerate(line_fields):
            lineObj[field] = values[i]
        lines[lineObj['lineID']] = lineObj

In [56]:
lines['L1045']

{'lineID': 'L1045',
 'characterID': 'u0',
 'movieID': 'm0',
 'character': 'BIANCA',
 'text': 'They do not!\n'}

In [51]:
conv_fields=['character1ID','character2ID','movieID','utteranceIDs']
conversations=[]
with open(conv_filepath,'r') as f:
    for line in f:
        values = line.split(" +++$+++ ")
        convObj = {}
        for i,fields in enumerate(conv_fields):
            convObj[fields]= values[i]
        lineIds = eval(convObj['utteranceIDs'])
        convObj['lines']=[]
        for lineID in lineIds:
            convObj['lines'].append(lines[lineID])
        conversations.append(convObj)

In [57]:
conversations[0]

{'character1ID': 'u0',
 'character2ID': 'u2',
 'movieID': 'm0',
 'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n",
 'lines': [{'lineID': 'L194',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
  {'lineID': 'L195',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
  {'lineID': 'L196',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
  {'lineID': 'L197',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}]}

In [58]:
conversations[0]

{'character1ID': 'u0',
 'character2ID': 'u2',
 'movieID': 'm0',
 'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n",
 'lines': [{'lineID': 'L194',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
  {'lineID': 'L195',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
  {'lineID': 'L196',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
  {'lineID': 'L197',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}]}

In [72]:
#extracting paris of sentences from conversations
qa_pairs=[]
for conversation in conversations:
    #iterating over all the lines in the conversation
    for i in range(len(conversation['lines']) - 1):
        inputLine = conversation['lines'][i]['text'].strip()
        targettLine = conversation['lines'][i+1]['text'].strip()
        if inputLine and targettLine:
            qa_pairs.append([inputLine,targettLine])

In [100]:
datafile = os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
delimiter = '\t'
delimiter = codecs.encode(delimiter,'unicode_escape')
with open(datafile,'w',encoding ='utf-8') as file:
    writer = csv.writer(file,delimiter)
    for pair in qa_pairs:
        writer.writerow(pair)
        
print("Done writing in file")

Done writing in file


In [89]:
with open(datafile,'rb') as file:
    lines = file.readlines()
for line in lines[:9]:
    print(line)

b'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.,"Well, I thought we\'d start with pronunciation, if that\'s okay with you."\r\r\n'
b'"Well, I thought we\'d start with pronunciation, if that\'s okay with you.",Not the hacking and gagging and spitting part.  Please.\r\r\n'
b"Not the hacking and gagging and spitting part.  Please.,Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\r\n"
b"You're asking me out.  That's so cute. What's your name again?,Forget it.\r\r\n"
b'"No, no, it\'s my fault -- we didn\'t have a proper introduction ---",Cameron.\r\r\n'
b'Cameron.,"The thing is, Cameron -- I\'m at the mercy of a particularly hideous breed of loser.  My sister.  I can\'t date until she does."\r\r\n'
b'"The thing is, Cameron -- I\'m at the mercy of a particularly hideous breed of loser.  My sister.  I can\'t date until she does.",Seems like she could get a date easy enough...\r\r

In [91]:
#processing the words
PAD_token = 0  # used for padding short sentences
SOS_token = 1  #start of sentence token <START>
EOS_token = 2  # end of sentence token <END>


class Vocabulary:
    def __init__(self,name):
        self.name = name
        self.word2index = {}  #indexing the words
        self.word2count = {}  #frequency of words
        self.index2word = {PAD_token:"PAD",SOS_token :  "SOS",EOS_token : "EOS"}  #initializing the indexes with PAD,SOS and EOS
        self.num_words = 3  # already exisiting previous words in index and the size
    
    def addSentence(self,sentence):
        for word in sentence.split(" "):
            self.addWord(word)
            
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words +=1
        else:
            self.word2count[word] +=1
            
    #removing words below a certain threshold
    def trim(self,min_count):
        keep_words=[]
        for word,frequency in self.word2count.items():
            if frequency >= min_count :
                keep_words.append(word)
        print("keep words {}/{} = {:.4f}".format(len(keep_words) , len(self.word2count),len(keep_words)/len(self.word2count)))
        #reinitializing the dictionaries
        self.word2index={}
        self.word2count ={}
        self.index2word ={PAD_token:"PAD",SOS_token :  "SOS",EOS_token : "EOS"}
        self.num_words =2
        for word in keep_words:
            self.addWord(word)

In [92]:
def unicodeToAscii(sentence):
    return ''.join(word for word in unicodedata.normalize('NFD',sentence) if unicodedata.category(word) != 'Mn')

In [94]:
#Lowercase, trim white spaces, lines ... etc and removing non-letter characters
def normalizeString(string_):
    s = unicodeToAscii(string_.lower().strip())
    s = re.sub(r"([.!?])",r" \1",s)
    s = re.sub(r"[^a-zA-Z.!?]+",r" ",s)
    s = re.sub(r"\s+",r" ",s).strip()
    return s

In [95]:
normalizeString("aaa123!s's   dd?")

'aaa !s s dd ?'

In [101]:
datafile = os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
print("Reading and processing the file ....")
lines = open(datafile,encoding='utf-8').read().strip().split('\n')
pairs = [[normalizeString(s) for s in pair.split('\t')] for pair in lines]
print("Done Reading")
voc = Vocabulary("cornell movie-dialogue corpus")

Reading and processing the file ....
Done Reading


In [110]:
pairs

[['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again . well i thought we d start with pronunciation if that s okay with you .'],
 [''],
 ['well i thought we d start with pronunciation if that s okay with you . not the hacking and gagging and spitting part . please .'],
 [''],
 ['not the hacking and gagging and spitting part . please . okay . . . then how bout we try out some french cuisine . saturday ? night ?'],
 [''],
 ['you re asking me out . that s so cute . what s your name again ? forget it .'],
 [''],
 ['no no it s my fault we didn t have a proper introduction cameron .'],
 [''],
 ['cameron . the thing is cameron i m at the mercy of a particularly hideous breed of loser . my sister . i can t date until she does .'],
 [''],
 ['the thing is cameron i m at the mercy of a particularly hideous breed of loser . my sister . i can t date until she does . seems like she could get a date easy enough . . .'],

In [111]:
# returns true if both the sentences in a single pair p are under the max length threshold
MAX_LENGTH = 10
def filterPair(p):
    return len(p[0].split()) <MAX_LENGTH and len(p[1].split()) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]